In [7]:
from json import load, dump
import os
import re
import numpy as np
import pandas as pd

In [8]:
results = []

string = "</think>"

for file in os.listdir("EXTRACTED_FILES"):
	try:
		with open(f"EXTRACTED_FILES/{file}", "r", encoding="utf-8") as f:
			model = load(f)
	except:
		print(file)
		continue
	cnt_plus = 0
	cnt_neg = 0
	for instance in model:
		index = instance["raw_model_output"].find(string)
		if index == -1:
			output = instance["raw_model_output"]
		else:
			output = instance["raw_model_output"][index + len(string):]
		option = re.findall(r'\d+', output)
		option = [int(x) for x in option if int(x) in range(5)]
		if len(option) == 1:
			instance["predicted_label"] = option[0]
		else:
			instance["predicted_label"] = -1
			if len(option) > 1:
				cnt_plus += 1
			else:
				cnt_neg += 1
	with open(f"new_method/{file}", "w", encoding="utf-8") as f:
		dump(model, f)
	results.append({"model": file, "more than one number": cnt_plus, "no valid numbers": cnt_neg, "more than one number %": 100 * cnt_plus / len(model), "no valid numbers %": 100 * cnt_neg / len(model)})

df = pd.DataFrame(results)

In [9]:
df

,model,more than one number,no valid numbers,more than one number %,no valid numbers %
0,qwen3_14b_semeval,19,12,0.153486,0.096938
1,qwen3_8b_semeval,4,4,0.032313,0.032313
2,gpt-oss_20b_semeval,2,8,0.016156,0.064626
3,llama3.2_latest_semeval,0,114,0.000000,0.920914
4,gemma2_2b_sst,0,0,0.000000,0.000000
...,...,...,...,...,...
56,gemma3_27b_semeval,0,0,0.000000,0.000000
57,qwen3_30b_semeval,27,0,0.218111,0.000000
58,phi4-mini_3.8b_semeval,10555,548,85.265369,4.426852
59,phi3.5_3.8b_sst,639,737,28.914027,33.348416
